In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
shared_path = '/content/drive/Shareddrives/Baiying'
os.chdir(shared_path)
print("Working directory:", os.getcwd())

%cd chronos-forecasting

Mounted at /content/drive
Working directory: /content/drive/Shareddrives/Baiying
/content/drive/Shareddrives/Baiying/chronos-forecasting


In [2]:
!pip install 'chronos-forecasting[extras]>=2.2' 'matplotlib'

INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of aiobotocore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/

In [3]:
import os

# Use only 1 GPU if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from chronos import BaseChronosPipeline, Chronos2Pipeline

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported using device_map="cpu"
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained("amazon/chronos-2", device_map="cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error

def load_and_prepare_data(file_path):
    """Load and prepare the glucose monitoring data."""
    context_df = pd.read_csv(file_path)
    df = context_df.copy()
    df = df.rename(columns={'BGvalue': 'target'})
    df['item_id'] = 'patient_1'
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    df = df[['item_id', 'timestamp', 'target']]
    return df

def split_into_sequences(df, gap_threshold_hours=1):
    """Split data into continuous sequences based on time gaps."""
    df['time_diff'] = df['timestamp'].diff()
    gap_threshold = pd.Timedelta(hours=gap_threshold_hours)

    df['new_sequence'] = (df['time_diff'] > gap_threshold) | (df['time_diff'].isna())
    df['sequence_id'] = df['new_sequence'].cumsum()

    sequences = []
    for seq_id, group in df.groupby('sequence_id'):
        group = group.drop(columns=['time_diff', 'new_sequence', 'sequence_id']).reset_index(drop=True)
        sequences.append(group)

    return sequences

def rolling_window_forecast(sequences, pipeline, context_length, prediction_length=18, step_size=1, verbose=False):
    """
    Perform rolling window forecasting across all sequences.

    Parameters:
    -----------
    sequences : list of DataFrames
        List of continuous data sequences
    pipeline : Chronos2Pipeline
        The forecasting pipeline
    context_length : int
        Number of historical time steps to use as context
    prediction_length : int
        Number of future time steps to predict
    step_size : int
        Step size for rolling window
    verbose : bool
        Whether to print progress

    Returns:
    --------
    all_predictions : list
        List of prediction arrays
    all_ground_truth : list
        List of ground truth arrays
    all_timestamps : list
        List of timestamp arrays
    all_sequence_ids : list
        List of sequence IDs for each prediction
    """
    all_predictions = []
    all_ground_truth = []
    all_timestamps = []
    all_sequence_ids = []

    total_windows = 0

    for seq_idx, seq_df in enumerate(sequences):
        seq_length = len(seq_df)
        max_start_idx = seq_length - context_length - prediction_length

        if max_start_idx < 0:
            if verbose:
                print(f"    Seq {seq_idx+1}: Too short ({seq_length} points), skipping...")
            continue

        num_windows = max_start_idx + 1
        total_windows += num_windows

        for start_idx in range(0, max_start_idx + 1, step_size):
            end_idx = start_idx + context_length
            pred_end_idx = end_idx + prediction_length

            context_window = seq_df.iloc[start_idx:end_idx].copy()
            ground_truth_window = seq_df.iloc[end_idx:pred_end_idx].copy()

            try:
                pred_df = pipeline.predict_df(
                    context_window,
                    prediction_length=prediction_length,
                    quantile_levels=[0.1, 0.5, 0.9]
                )

                predictions = pred_df[pred_df['target_name'] == 'target']['predictions'].values
                ground_truth = ground_truth_window['target'].values
                timestamps = ground_truth_window['timestamp'].values

                all_predictions.append(predictions)
                all_ground_truth.append(ground_truth)
                all_timestamps.append(timestamps)
                all_sequence_ids.append(seq_idx)

            except Exception as e:
                if verbose:
                    print(f"    Error at window {start_idx} in seq {seq_idx+1}: {e}")
                continue

    return all_predictions, all_ground_truth, all_timestamps, all_sequence_ids

def calculate_metrics(all_predictions, all_ground_truth, horizon_steps):
    """Calculate RMSE and MAE for different prediction horizons."""
    results = {}

    for horizon_name, horizon_step in horizon_steps.items():
        rmse_values = []
        mae_values = []

        for pred, gt in zip(all_predictions, all_ground_truth):
            if len(pred) >= horizon_step and len(gt) >= horizon_step:
                pred_value = pred[horizon_step - 1]
                gt_value = gt[horizon_step - 1]

                squared_error = (pred_value - gt_value) ** 2
                absolute_error = abs(pred_value - gt_value)

                rmse_values.append(squared_error)
                mae_values.append(absolute_error)

        if len(rmse_values) > 0:
            avg_rmse = np.sqrt(np.mean(rmse_values))
            avg_mae = np.mean(mae_values)

            results[horizon_name] = {
                'RMSE': avg_rmse,
                'MAE': avg_mae,
                'n_samples': len(rmse_values)
            }
        else:
            results[horizon_name] = {
                'RMSE': np.nan,
                'MAE': np.nan,
                'n_samples': 0
            }

    return results

def evaluate_single_patient(file_path, pipeline, context_lengths, prediction_length=18, step_size=1):
    """
    Evaluate a single patient across multiple context lengths.

    Returns:
    --------
    patient_results : dict
        Dictionary with results for each context length
    """
    horizon_steps = {
        '15min': 3,
        '30min': 6,
        '60min': 12,
        '90min': 18
    }

    patient_name = Path(file_path).stem

    try:
        # Load and prepare data
        df = load_and_prepare_data(file_path)

        # Split into sequences
        sequences = split_into_sequences(df, gap_threshold_hours=1)

        print(f"  Patient: {patient_name}")
        print(f"    Total points: {len(df)}, Sequences: {len(sequences)}")

        patient_results = {}

        # Evaluate for each context length
        for context_length in context_lengths:
            # Perform rolling window forecasting
            all_predictions, all_ground_truth, all_timestamps, all_sequence_ids = rolling_window_forecast(
                sequences, pipeline, context_length, prediction_length, step_size, verbose=False
            )

            if len(all_predictions) == 0:
                print(f"    Context {context_length}: No predictions (insufficient data)")
                patient_results[context_length] = None
                continue

            # Calculate metrics
            results = calculate_metrics(all_predictions, all_ground_truth, horizon_steps)
            patient_results[context_length] = results

            print(f"    Context {context_length}: {len(all_predictions)} predictions")

        return patient_results

    except Exception as e:
        print(f"  Error processing {patient_name}: {e}")
        return None

def aggregate_results_across_patients(all_patient_results, context_lengths):
    """
    Aggregate results across all patients.

    Returns:
    --------
    aggregated_results : DataFrame
        Summary table with average metrics across all patients
    """
    horizons = ['15min', '30min', '60min', '90min']

    summary_data = []

    for context_length in context_lengths:
        for horizon in horizons:
            rmse_values = []
            mae_values = []

            # Collect metrics from all patients for this context length and horizon
            for patient_name, patient_results in all_patient_results.items():
                if patient_results is not None and context_length in patient_results:
                    if patient_results[context_length] is not None:
                        if horizon in patient_results[context_length]:
                            r = patient_results[context_length][horizon]
                            if not np.isnan(r['RMSE']):
                                rmse_values.append(r['RMSE'])
                                mae_values.append(r['MAE'])

            # Calculate average across patients
            if len(rmse_values) > 0:
                avg_rmse = np.mean(rmse_values)
                std_rmse = np.std(rmse_values)
                avg_mae = np.mean(mae_values)
                std_mae = np.std(mae_values)
                n_patients = len(rmse_values)
            else:
                avg_rmse = np.nan
                std_rmse = np.nan
                avg_mae = np.nan
                std_mae = np.nan
                n_patients = 0

            summary_data.append({
                'Context_Length': context_length,
                'Context_Hours': context_length * 5 / 60,
                'Horizon': horizon,
                'RMSE_Mean': avg_rmse,
                'RMSE_Std': std_rmse,
                'MAE_Mean': avg_mae,
                'MAE_Std': std_mae,
                'N_Patients': n_patients
            })

    return pd.DataFrame(summary_data)

def save_detailed_results(all_patient_results, context_lengths, output_dir='./results'):
    """Save detailed per-patient results to CSV files."""
    os.makedirs(output_dir, exist_ok=True)

    horizons = ['15min', '30min', '60min', '90min']

    for context_length in context_lengths:
        patient_data = []

        for patient_name, patient_results in all_patient_results.items():
            if patient_results is not None and context_length in patient_results:
                if patient_results[context_length] is not None:
                    for horizon in horizons:
                        if horizon in patient_results[context_length]:
                            r = patient_results[context_length][horizon]
                            patient_data.append({
                                'Patient': patient_name,
                                'Context_Length': context_length,
                                'Horizon': horizon,
                                'RMSE': r['RMSE'],
                                'MAE': r['MAE'],
                                'N_Samples': r['n_samples']
                            })

        if patient_data:
            detail_df = pd.DataFrame(patient_data)
            output_file = f"{output_dir}/context_{context_length}_detailed.csv"
            detail_df.to_csv(output_file, index=False)
            print(f"Saved: {output_file}")


In [ ]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

# Directory containing all patient CSV files
data_dir = "/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset/BIG_IDEA_LAB"

# Get all CSV files
csv_files = sorted(Path(data_dir).glob("*.csv"))
print(f"Found {len(csv_files)} patient files\n")

# Context lengths to evaluate
context_lengths = [12, 48, 96, 144, 192, 288]

# Storage for all patient results
all_patient_results = {}

# Process each patient
print("="*70)
print("PROCESSING ALL PATIENTS")
print("="*70)

for i, file_path in enumerate(csv_files, 1):
    patient_name = file_path.stem
    print(f"\n[{i}/{len(csv_files)}] Processing {patient_name}...")

    patient_results = evaluate_single_patient(
        file_path=str(file_path),
        pipeline=pipeline,
        context_lengths=context_lengths,
        prediction_length=18,
        step_size=1
    )

    all_patient_results[patient_name] = patient_results

# =============================================================================
# AGGREGATE RESULTS
# =============================================================================

print("\n" + "="*70)
print("AGGREGATING RESULTS ACROSS ALL PATIENTS")
print("="*70)

summary_df = aggregate_results_across_patients(all_patient_results, context_lengths)

# =============================================================================
# DISPLAY SUMMARY TABLE
# =============================================================================

print("\n" + "="*90)
print("SUMMARY: AVERAGE PERFORMANCE ACROSS ALL PATIENTS")
print("="*90)

# Print by horizon
for horizon in ['15min', '30min', '60min', '90min']:
    print(f"\n{horizon.upper()} Prediction Horizon:")
    print("-"*90)
    print(f"{'Context':<10} {'Hours':<10} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
    print("-"*90)

    horizon_data = summary_df[summary_df['Horizon'] == horizon]
    for _, row in horizon_data.iterrows():
        rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
        mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
        print(f"{int(row['Context_Length']):<10} {row['Context_Hours']:<10.1f} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

print("\n" + "="*90)

# =============================================================================
# CREATE PIVOT TABLES FOR EASIER VIEWING
# =============================================================================

# Pivot table for RMSE
print("\n" + "="*90)
print("RMSE SUMMARY TABLE (Mean ± Std)")
print("="*90)
rmse_pivot = summary_df.pivot_table(
    index='Context_Length',
    columns='Horizon',
    values='RMSE_Mean',
    aggfunc='first'
)
rmse_std_pivot = summary_df.pivot_table(
    index='Context_Length',
    columns='Horizon',
    values='RMSE_Std',
    aggfunc='first'
)

for context_length in context_lengths:
    if context_length in rmse_pivot.index:
        row_str = f"{context_length:<15} ({context_length*5/60:.1f}h)  "
        for horizon in ['15min', '30min', '60min', '90min']:
            if horizon in rmse_pivot.columns:
                mean_val = rmse_pivot.loc[context_length, horizon]
                std_val = rmse_std_pivot.loc[context_length, horizon]
                if not np.isnan(mean_val):
                    row_str += f"{mean_val:.2f}±{std_val:.2f}  "
                else:
                    row_str += "N/A          "
        print(row_str)

# Pivot table for MAE
print("\n" + "="*90)
print("MAE SUMMARY TABLE (Mean ± Std)")
print("="*90)
mae_pivot = summary_df.pivot_table(
    index='Context_Length',
    columns='Horizon',
    values='MAE_Mean',
    aggfunc='first'
)
mae_std_pivot = summary_df.pivot_table(
    index='Context_Length',
    columns='Horizon',
    values='MAE_Std',
    aggfunc='first'
)

print(f"{'Context (Hours)':<25} {'15min':<18} {'30min':<18} {'60min':<18} {'90min':<18}")
print("-"*90)
for context_length in context_lengths:
    if context_length in mae_pivot.index:
        row_str = f"{context_length:<15} ({context_length*5/60:.1f}h)  "
        for horizon in ['15min', '30min', '60min', '90min']:
            if horizon in mae_pivot.columns:
                mean_val = mae_pivot.loc[context_length, horizon]
                std_val = mae_std_pivot.loc[context_length, horizon]
                if not np.isnan(mean_val):
                    row_str += f"{mean_val:.2f}±{std_val:.2f}  "
                else:
                    row_str += "N/A          "
        print(row_str)

print("\n" + "="*90)

# =============================================================================
# SAVE RESULTS
# =============================================================================

# Save summary table
summary_df.to_csv('chronos2_summary_all_patients.csv', index=False)
print("\nSummary saved to: chronos2_summary_all_patients.csv")

# Save detailed per-patient results
save_detailed_results(all_patient_results, context_lengths, output_dir='./patient_results')

print("\nEvaluation complete!")

Found 16 patient files

PROCESSING ALL PATIENTS

[1/16] Processing Dexcom_001...
  Patient: Dexcom_001
    Total points: 520, Sequences: 1
    Context 12: 491 predictions
    Context 48: 455 predictions
    Context 96: 407 predictions
    Context 144: 359 predictions
    Context 192: 311 predictions
    Context 288: 215 predictions

[2/16] Processing Dexcom_002...
  Patient: Dexcom_002
    Total points: 458, Sequences: 1
    Context 12: 429 predictions
    Context 48: 393 predictions
    Context 96: 345 predictions
    Context 144: 297 predictions
    Context 192: 249 predictions
    Context 288: 153 predictions

[3/16] Processing Dexcom_003...
  Patient: Dexcom_003
    Total points: 463, Sequences: 1
    Context 12: 434 predictions
    Context 48: 398 predictions
    Context 96: 350 predictions
    Context 144: 302 predictions
    Context 192: 254 predictions
    Context 288: 158 predictions

[4/16] Processing Dexcom_004...
  Patient: Dexcom_004
    Total points: 452, Sequences: 1
   

In [ ]:
# =============================================================================
# MAIN EXECUTION - MULTI-DATASET EVALUATION
# =============================================================================

# Base directory containing dataset folders
base_dir = "/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset"

# List of dataset folders to process
dataset_folders = [
    "BIG_IDEA_LAB",
    "ShanghaiT1DM",
    "ShanghaiT2DM",
    "CGMacros",
    "UCHTT1DM",
    # "1_Hall2018",
    # "2_D1NAMO",
    # "3_cOLAS2019",
    # "14_HUPA-UCM",
    # "17_T1DM-UOM",
    # "18_Bris-T1D Open",
    # "19_AZT1D"
    # Add more dataset names here as needed
]

# Context lengths to evaluate
context_lengths = [12, 48, 96, 144, 192, 288]

# Storage for all datasets' results
all_datasets_results = {}

# =============================================================================
# PROCESS EACH DATASET
# =============================================================================

print("="*70)
print("PROCESSING MULTIPLE DATASETS")
print("="*70)

for dataset_idx, dataset_name in enumerate(dataset_folders, 1):
    data_dir = os.path.join(base_dir, dataset_name)

    # Check if directory exists
    if not os.path.exists(data_dir):
        print(f"\n[{dataset_idx}/{len(dataset_folders)}] Dataset '{dataset_name}' not found at {data_dir}, skipping...")
        continue

    print(f"\n{'='*70}")
    print(f"[{dataset_idx}/{len(dataset_folders)}] PROCESSING DATASET: {dataset_name}")
    print(f"{'='*70}")

    # Get all CSV files
    csv_files = sorted(Path(data_dir).glob("*.csv"))
    print(f"Found {len(csv_files)} patient files")

    if len(csv_files) == 0:
        print(f"No CSV files found in {dataset_name}, skipping...")
        continue

    # Storage for this dataset's patient results
    dataset_patient_results = {}

    # Process each patient
    for i, file_path in enumerate(csv_files, 1):
        patient_name = file_path.stem
        print(f"\n  [{i}/{len(csv_files)}] Processing {patient_name}...")

        patient_results = evaluate_single_patient(
            file_path=str(file_path),
            pipeline=pipeline,
            context_lengths=context_lengths,
            prediction_length=18,
            step_size=1
        )

        dataset_patient_results[patient_name] = patient_results

    # Store results for this dataset
    all_datasets_results[dataset_name] = dataset_patient_results

    # Aggregate results for this dataset
    print(f"\n{'='*70}")
    print(f"AGGREGATING RESULTS FOR {dataset_name}")
    print(f"{'='*70}")

    summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)

    # Display summary for this dataset
    print(f"\n{'='*90}")
    print(f"SUMMARY: {dataset_name} - AVERAGE PERFORMANCE ACROSS ALL PATIENTS")
    print(f"{'='*90}")

    # Print by horizon
    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Prediction Horizon:")
        print("-"*90)
        print(f"{'Context':<10} {'Hours':<10} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
        print("-"*90)

        horizon_data = summary_df[summary_df['Horizon'] == horizon]
        for _, row in horizon_data.iterrows():
            rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
            mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
            print(f"{int(row['Context_Length']):<10} {row['Context_Hours']:<10.1f} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

    print(f"\n{'='*90}")

    # Save dataset-specific results
    output_dir = f'./results_{dataset_name}'
    os.makedirs(output_dir, exist_ok=True)

    summary_df.to_csv(f'{output_dir}/chronos2_summary_{dataset_name}.csv', index=False)
    save_detailed_results(dataset_patient_results, context_lengths, output_dir=output_dir)

    print(f"\n{dataset_name} results saved to: {output_dir}/")

# =============================================================================
# CROSS-DATASET COMPARISON
# =============================================================================

print("\n" + "="*90)
print("CROSS-DATASET COMPARISON")
print("="*90)

# Create comparison dataframes
comparison_data = []

for dataset_name, dataset_patient_results in all_datasets_results.items():
    summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)
    summary_df['Dataset'] = dataset_name
    comparison_data.append(summary_df)

if comparison_data:
    combined_df = pd.concat(comparison_data, ignore_index=True)

    # Display comparison for each horizon and context length
    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{'='*90}")
        print(f"{horizon.upper()} PREDICTION HORIZON - CROSS-DATASET COMPARISON")
        print(f"{'='*90}")

        for context_length in context_lengths:
            print(f"\nContext Length: {context_length} ({context_length*5/60:.1f} hours)")
            print("-"*90)
            print(f"{'Dataset':<20} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
            print("-"*90)

            subset = combined_df[
                (combined_df['Horizon'] == horizon) &
                (combined_df['Context_Length'] == context_length)
            ]

            for _, row in subset.iterrows():
                rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
                mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
                print(f"{row['Dataset']:<20} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

    # Save combined results
    combined_df.to_csv('chronos2_combined_all_datasets.csv', index=False)
    print(f"\n{'='*90}")
    print("Combined results saved to: chronos2_combined_all_datasets.csv")

    # =============================================================================
    # CREATE COMPARISON PIVOT TABLES
    # =============================================================================

    print(f"\n{'='*90}")
    print("RMSE COMPARISON ACROSS DATASETS")
    print(f"{'='*90}")

    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Horizon:")
        print("-"*90)

        horizon_data = combined_df[combined_df['Horizon'] == horizon]
        pivot = horizon_data.pivot_table(
            index='Dataset',
            columns='Context_Length',
            values='RMSE_Mean',
            aggfunc='first'
        )

        print(f"\n{'Dataset':<20}", end='')
        for ctx in context_lengths:
            if ctx in pivot.columns:
                print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
        print()
        print("-"*90)

        for dataset in pivot.index:
            print(f"{dataset:<20}", end='')
            for ctx in context_lengths:
                if ctx in pivot.columns:
                    val = pivot.loc[dataset, ctx]
                    if not np.isnan(val):
                        print(f"{val:.2f}".ljust(15), end='')
                    else:
                        print("N/A".ljust(15), end='')
            print()

    print(f"\n{'='*90}")
    print("MAE COMPARISON ACROSS DATASETS")
    print(f"{'='*90}")

    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Horizon:")
        print("-"*90)

        horizon_data = combined_df[combined_df['Horizon'] == horizon]
        pivot = horizon_data.pivot_table(
            index='Dataset',
            columns='Context_Length',
            values='MAE_Mean',
            aggfunc='first'
        )

        print(f"\n{'Dataset':<20}", end='')
        for ctx in context_lengths:
            if ctx in pivot.columns:
                print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
        print()
        print("-"*90)

        for dataset in pivot.index:
            print(f"{dataset:<20}", end='')
            for ctx in context_lengths:
                if ctx in pivot.columns:
                    val = pivot.loc[dataset, ctx]
                    if not np.isnan(val):
                        print(f"{val:.2f}".ljust(15), end='')
                    else:
                        print("N/A".ljust(15), end='')
            print()

print("\n" + "="*90)
print("EVALUATION COMPLETE FOR ALL DATASETS!")
print("="*90)

PROCESSING MULTIPLE DATASETS

[1/5] PROCESSING DATASET: BIG_IDEA_LAB
Found 16 patient files

  [1/16] Processing Dexcom_001...
  Patient: Dexcom_001
    Total points: 513, Sequences: 1
    Context 12: 484 predictions
    Context 48: 448 predictions
    Context 96: 400 predictions
    Context 144: 352 predictions
    Context 192: 304 predictions
    Context 288: 208 predictions

  [2/16] Processing Dexcom_002...
  Patient: Dexcom_002
    Total points: 424, Sequences: 1
    Context 12: 395 predictions
    Context 48: 359 predictions
    Context 96: 311 predictions
    Context 144: 263 predictions
    Context 192: 215 predictions
    Context 288: 119 predictions

  [3/16] Processing Dexcom_003...
  Patient: Dexcom_003
    Total points: 461, Sequences: 1
    Context 12: 432 predictions
    Context 48: 396 predictions
    Context 96: 348 predictions
    Context 144: 300 predictions
    Context 192: 252 predictions
    Context 288: 156 predictions

  [4/16] Processing Dexcom_004...
  Patient

/tmp/ipython-input-3025122032.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'])


  Patient: Dexcom_016
    Total points: 457, Sequences: 1
    Context 12: 428 predictions
    Context 48: 392 predictions
    Context 96: 344 predictions
    Context 144: 296 predictions
    Context 192: 248 predictions
    Context 288: 152 predictions

AGGREGATING RESULTS FOR BIG_IDEA_LAB

SUMMARY: BIG_IDEA_LAB - AVERAGE PERFORMANCE ACROSS ALL PATIENTS

15MIN Prediction Horizon:
------------------------------------------------------------------------------------------
Context    Hours      RMSE (Mean±Std)           MAE (Mean±Std)            N Patients     
------------------------------------------------------------------------------------------
12         1.0        9.00±1.76                 5.83±1.08                 16             
48         4.0        9.00±1.92                 5.76±1.11                 16             
96         8.0        8.56±1.69                 5.53±0.99                 16             
144        12.0       8.05±1.45                 5.26±0.89                 1

In [5]:
# =============================================================================
# MAIN EXECUTION - MULTI-DATASET EVALUATION
# =============================================================================

# Base directory containing dataset folders
base_dir = "/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset/controlled_datasets"

# List of dataset folders to process
dataset_folders = [
    # "OhioT1DM",
    # "8_DiaTrend",
    "5_T1DEXI"
    # Add more dataset names here as needed
]

# Context lengths to evaluate
# context_lengths = [12, 48, 96, 144, 192, 288]
context_lengths = [144]

# Storage for all datasets' results
all_datasets_results = {}

# =============================================================================
# PROCESS EACH DATASET
# =============================================================================

print("="*70)
print("PROCESSING MULTIPLE DATASETS")
print("="*70)

for dataset_idx, dataset_name in enumerate(dataset_folders, 1):
    data_dir = os.path.join(base_dir, dataset_name)

    # Check if directory exists
    if not os.path.exists(data_dir):
        print(f"\n[{dataset_idx}/{len(dataset_folders)}] Dataset '{dataset_name}' not found at {data_dir}, skipping...")
        continue

    print(f"\n{'='*70}")
    print(f"[{dataset_idx}/{len(dataset_folders)}] PROCESSING DATASET: {dataset_name}")
    print(f"{'='*70}")

    # Get all CSV files
    csv_files = sorted(Path(data_dir).glob("*.csv"))
    print(f"Found {len(csv_files)} patient files")

    if len(csv_files) == 0:
        print(f"No CSV files found in {dataset_name}, skipping...")
        continue

    # Storage for this dataset's patient results
    dataset_patient_results = {}

    # Process each patient
    for i, file_path in enumerate(csv_files, 1):
        patient_name = file_path.stem
        print(f"\n  [{i}/{len(csv_files)}] Processing {patient_name}...")

        patient_results = evaluate_single_patient(
            file_path=str(file_path),
            pipeline=pipeline,
            context_lengths=context_lengths,
            prediction_length=18,
            step_size=1
        )

        dataset_patient_results[patient_name] = patient_results

    # Store results for this dataset
    all_datasets_results[dataset_name] = dataset_patient_results

    # Aggregate results for this dataset
    print(f"\n{'='*70}")
    print(f"AGGREGATING RESULTS FOR {dataset_name}")
    print(f"{'='*70}")

    summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)

    # Display summary for this dataset
    print(f"\n{'='*90}")
    print(f"SUMMARY: {dataset_name} - AVERAGE PERFORMANCE ACROSS ALL PATIENTS")
    print(f"{'='*90}")

    # Print by horizon
    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Prediction Horizon:")
        print("-"*90)
        print(f"{'Context':<10} {'Hours':<10} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
        print("-"*90)

        horizon_data = summary_df[summary_df['Horizon'] == horizon]
        for _, row in horizon_data.iterrows():
            rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
            mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
            print(f"{int(row['Context_Length']):<10} {row['Context_Hours']:<10.1f} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

    print(f"\n{'='*90}")

    # Save dataset-specific results
    output_dir = f'./results_{dataset_name}'
    os.makedirs(output_dir, exist_ok=True)

    summary_df.to_csv(f'{output_dir}/chronos2_summary_{dataset_name}.csv', index=False)
    save_detailed_results(dataset_patient_results, context_lengths, output_dir=output_dir)

    print(f"\n{dataset_name} results saved to: {output_dir}/")

# =============================================================================
# CROSS-DATASET COMPARISON
# =============================================================================

print("\n" + "="*90)
print("CROSS-DATASET COMPARISON")
print("="*90)

# Create comparison dataframes
comparison_data = []

for dataset_name, dataset_patient_results in all_datasets_results.items():
    summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)
    summary_df['Dataset'] = dataset_name
    comparison_data.append(summary_df)

if comparison_data:
    combined_df = pd.concat(comparison_data, ignore_index=True)

    # Display comparison for each horizon and context length
    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{'='*90}")
        print(f"{horizon.upper()} PREDICTION HORIZON - CROSS-DATASET COMPARISON")
        print(f"{'='*90}")

        for context_length in context_lengths:
            print(f"\nContext Length: {context_length} ({context_length*5/60:.1f} hours)")
            print("-"*90)
            print(f"{'Dataset':<20} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
            print("-"*90)

            subset = combined_df[
                (combined_df['Horizon'] == horizon) &
                (combined_df['Context_Length'] == context_length)
            ]

            for _, row in subset.iterrows():
                rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
                mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
                print(f"{row['Dataset']:<20} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

    # Save combined results
    combined_df.to_csv('./chronos2_combined_all_datasets_controlled.csv', index=False)
    print(f"\n{'='*90}")
    print("Combined results saved to: chronos2_combined_all_datasets_controlled.csv")

    # =============================================================================
    # CREATE COMPARISON PIVOT TABLES
    # =============================================================================

    print(f"\n{'='*90}")
    print("RMSE COMPARISON ACROSS DATASETS")
    print(f"{'='*90}")

    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Horizon:")
        print("-"*90)

        horizon_data = combined_df[combined_df['Horizon'] == horizon]
        pivot = horizon_data.pivot_table(
            index='Dataset',
            columns='Context_Length',
            values='RMSE_Mean',
            aggfunc='first'
        )

        print(f"\n{'Dataset':<20}", end='')
        for ctx in context_lengths:
            if ctx in pivot.columns:
                print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
        print()
        print("-"*90)

        for dataset in pivot.index:
            print(f"{dataset:<20}", end='')
            for ctx in context_lengths:
                if ctx in pivot.columns:
                    val = pivot.loc[dataset, ctx]
                    if not np.isnan(val):
                        print(f"{val:.2f}".ljust(15), end='')
                    else:
                        print("N/A".ljust(15), end='')
            print()

    print(f"\n{'='*90}")
    print("MAE COMPARISON ACROSS DATASETS")
    print(f"{'='*90}")

    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Horizon:")
        print("-"*90)

        horizon_data = combined_df[combined_df['Horizon'] == horizon]
        pivot = horizon_data.pivot_table(
            index='Dataset',
            columns='Context_Length',
            values='MAE_Mean',
            aggfunc='first'
        )

        print(f"\n{'Dataset':<20}", end='')
        for ctx in context_lengths:
            if ctx in pivot.columns:
                print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
        print()
        print("-"*90)

        for dataset in pivot.index:
            print(f"{dataset:<20}", end='')
            for ctx in context_lengths:
                if ctx in pivot.columns:
                    val = pivot.loc[dataset, ctx]
                    if not np.isnan(val):
                        print(f"{val:.2f}".ljust(15), end='')
                    else:
                        print("N/A".ljust(15), end='')
            print()

print("\n" + "="*90)
print("EVALUATION COMPLETE FOR ALL DATASETS!")
print("="*90)

PROCESSING MULTIPLE DATASETS

[1/1] PROCESSING DATASET: 5_T1DEXI
Found 502 patient files

  [1/502] Processing 1...
  Patient: 1
    Total points: 1580, Sequences: 1
    Context 144: 1419 predictions

  [2/502] Processing 1000...
  Patient: 1000
    Total points: 1584, Sequences: 2
    Context 144: 1262 predictions

  [3/502] Processing 1004...
  Patient: 1004
    Total points: 1595, Sequences: 2
    Context 144: 1273 predictions

  [4/502] Processing 1010...
  Patient: 1010
    Total points: 1541, Sequences: 1
    Context 144: 1380 predictions

  [5/502] Processing 1012...
  Patient: 1012
    Total points: 1565, Sequences: 2
    Context 144: 1256 predictions

  [6/502] Processing 1013...
  Patient: 1013
    Total points: 1548, Sequences: 1
    Context 144: 1387 predictions

  [7/502] Processing 1014...
  Patient: 1014
    Total points: 1558, Sequences: 3
    Context 144: 1200 predictions

  [8/502] Processing 1015...
  Patient: 1015
    Total points: 1499, Sequences: 4
    Context 144

/tmp/ipython-input-3025122032.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['timestamp'] = pd.to_datetime(df['timestamp'])


  Patient: 254
    Total points: 1446, Sequences: 2
    Context 144: 1124 predictions

  [269/502] Processing 255...
  Patient: 255
    Total points: 1505, Sequences: 2
    Context 144: 1183 predictions

  [270/502] Processing 256...
  Patient: 256
    Total points: 1591, Sequences: 1
    Context 144: 1430 predictions

  [271/502] Processing 261...
  Patient: 261
    Total points: 1582, Sequences: 2
    Context 144: 1260 predictions

  [272/502] Processing 263...
  Patient: 263
    Total points: 1501, Sequences: 5
    Context 144: 1013 predictions

  [273/502] Processing 267...
  Patient: 267
    Total points: 1590, Sequences: 2
    Context 144: 1268 predictions

  [274/502] Processing 270...
  Patient: 270
    Total points: 1574, Sequences: 2
    Context 144: 1280 predictions

  [275/502] Processing 284...
  Patient: 284
    Total points: 1575, Sequences: 1
    Context 144: 1414 predictions

  [276/502] Processing 288...
  Patient: 288
    Total points: 1509, Sequences: 4
    Context 

In [ ]:
def load_and_prepare_data_new(file_path):
    """Load and prepare the glucose monitoring data."""
    context_df = pd.read_csv(file_path)
    df = context_df.copy()
    df = df.rename(columns={'BGvalue': 'target'})
    df['item_id'] = 'patient_1'
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    df = df[['item_id', 'timestamp', 'target']]
    return df


def evaluate_single_patient(file_path, pipeline, context_lengths, prediction_length=18, step_size=1):
    """
    Evaluate a single patient across multiple context lengths.

    Returns:
    --------
    patient_results : dict
        Dictionary with results for each context length
    """
    horizon_steps = {
        '15min': 3,
        '30min': 6,
        '60min': 12,
        '90min': 18
    }

    patient_name = Path(file_path).stem

    try:
        # Load and prepare data
        df = load_and_prepare_data_new(file_path)

        # Split into sequences
        sequences = split_into_sequences(df, gap_threshold_hours=1)

        print(f"  Patient: {patient_name}")
        print(f"    Total points: {len(df)}, Sequences: {len(sequences)}")

        patient_results = {}

        # Evaluate for each context length
        for context_length in context_lengths:
            # Perform rolling window forecasting
            all_predictions, all_ground_truth, all_timestamps, all_sequence_ids = rolling_window_forecast(
                sequences, pipeline, context_length, prediction_length, step_size, verbose=False
            )

            if len(all_predictions) == 0:
                print(f"    Context {context_length}: No predictions (insufficient data)")
                patient_results[context_length] = None
                continue

            # Calculate metrics
            results = calculate_metrics(all_predictions, all_ground_truth, horizon_steps)
            patient_results[context_length] = results

            print(f"    Context {context_length}: {len(all_predictions)} predictions")

        return patient_results

    except Exception as e:
        print(f"  Error processing {patient_name}: {e}")
        return None

In [ ]:
# =============================================================================
# MAIN EXECUTION - MULTI-DATASET EVALUATION
# =============================================================================

# Base directory containing dataset folders
base_dir = "/content/drive/Shareddrives/Baiying/preprocessed_dataset/test_dataset"

# List of dataset folders to process
dataset_folders = [
    "1_Hall2018",
    # "2_D1NAMO",
    # "3_cOLAS2019",
    # "14_HUPA-UCM",
    # "17_T1DM-UOM",
    # "18_Bris-T1D Open",
    # "19_AZT1D"
    # Add more dataset names here as needed
]

# Context lengths to evaluate
context_lengths = [12, 48, 96, 144, 192, 288]

# Storage for all datasets' results
all_datasets_results = {}

# =============================================================================
# PROCESS EACH DATASET
# =============================================================================

print("="*70)
print("PROCESSING MULTIPLE DATASETS")
print("="*70)

for dataset_idx, dataset_name in enumerate(dataset_folders, 1):
    data_dir = os.path.join(base_dir, dataset_name)

    # Check if directory exists
    if not os.path.exists(data_dir):
        print(f"\n[{dataset_idx}/{len(dataset_folders)}] Dataset '{dataset_name}' not found at {data_dir}, skipping...")
        continue

    print(f"\n{'='*70}")
    print(f"[{dataset_idx}/{len(dataset_folders)}] PROCESSING DATASET: {dataset_name}")
    print(f"{'='*70}")

    # Get all CSV files
    csv_files = sorted(Path(data_dir).glob("*.csv"))
    print(f"Found {len(csv_files)} patient files")

    if len(csv_files) == 0:
        print(f"No CSV files found in {dataset_name}, skipping...")
        continue

    # Storage for this dataset's patient results
    dataset_patient_results = {}

    # Process each patient
    for i, file_path in enumerate(csv_files, 1):
        patient_name = file_path.stem
        print(f"\n  [{i}/{len(csv_files)}] Processing {patient_name}...")

        patient_results = evaluate_single_patient(
            file_path=str(file_path),
            pipeline=pipeline,
            context_lengths=context_lengths,
            prediction_length=18,
            step_size=1
        )

        dataset_patient_results[patient_name] = patient_results

    # Store results for this dataset
    all_datasets_results[dataset_name] = dataset_patient_results

    # Aggregate results for this dataset
    print(f"\n{'='*70}")
    print(f"AGGREGATING RESULTS FOR {dataset_name}")
    print(f"{'='*70}")

    summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)

    # Display summary for this dataset
    print(f"\n{'='*90}")
    print(f"SUMMARY: {dataset_name} - AVERAGE PERFORMANCE ACROSS ALL PATIENTS")
    print(f"{'='*90}")

    # Print by horizon
    for horizon in ['15min', '30min', '60min', '90min']:
        print(f"\n{horizon.upper()} Prediction Horizon:")
        print("-"*90)
        print(f"{'Context':<10} {'Hours':<10} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
        print("-"*90)

        horizon_data = summary_df[summary_df['Horizon'] == horizon]
        for _, row in horizon_data.iterrows():
            rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
            mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
            print(f"{int(row['Context_Length']):<10} {row['Context_Hours']:<10.1f} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

    print(f"\n{'='*90}")

    # Save dataset-specific results
    output_dir = f'./results_{dataset_name}'
    os.makedirs(output_dir, exist_ok=True)

    summary_df.to_csv(f'{output_dir}/chronos2_summary_{dataset_name}.csv', index=False)
    save_detailed_results(dataset_patient_results, context_lengths, output_dir=output_dir)

    print(f"\n{dataset_name} results saved to: {output_dir}/")

# # =============================================================================
# # CROSS-DATASET COMPARISON
# # =============================================================================

# print("\n" + "="*90)
# print("CROSS-DATASET COMPARISON")
# print("="*90)

# # Create comparison dataframes
# comparison_data = []

# for dataset_name, dataset_patient_results in all_datasets_results.items():
#     summary_df = aggregate_results_across_patients(dataset_patient_results, context_lengths)
#     summary_df['Dataset'] = dataset_name
#     comparison_data.append(summary_df)

# if comparison_data:
#     combined_df = pd.concat(comparison_data, ignore_index=True)

#     # Display comparison for each horizon and context length
#     for horizon in ['15min', '30min', '60min', '90min']:
#         print(f"\n{'='*90}")
#         print(f"{horizon.upper()} PREDICTION HORIZON - CROSS-DATASET COMPARISON")
#         print(f"{'='*90}")

#         for context_length in context_lengths:
#             print(f"\nContext Length: {context_length} ({context_length*5/60:.1f} hours)")
#             print("-"*90)
#             print(f"{'Dataset':<20} {'RMSE (Mean±Std)':<25} {'MAE (Mean±Std)':<25} {'N Patients':<15}")
#             print("-"*90)

#             subset = combined_df[
#                 (combined_df['Horizon'] == horizon) &
#                 (combined_df['Context_Length'] == context_length)
#             ]

#             for _, row in subset.iterrows():
#                 rmse_str = f"{row['RMSE_Mean']:.2f}±{row['RMSE_Std']:.2f}" if not np.isnan(row['RMSE_Mean']) else "N/A"
#                 mae_str = f"{row['MAE_Mean']:.2f}±{row['MAE_Std']:.2f}" if not np.isnan(row['MAE_Mean']) else "N/A"
#                 print(f"{row['Dataset']:<20} {rmse_str:<25} {mae_str:<25} {int(row['N_Patients']):<15}")

#     # Save combined results
#     combined_df.to_csv('chronos2_combined_all_datasets.csv', index=False)
#     print(f"\n{'='*90}")
#     print("Combined results saved to: chronos2_combined_all_datasets.csv")

#     # =============================================================================
#     # CREATE COMPARISON PIVOT TABLES
#     # =============================================================================

#     print(f"\n{'='*90}")
#     print("RMSE COMPARISON ACROSS DATASETS")
#     print(f"{'='*90}")

#     for horizon in ['15min', '30min', '60min', '90min']:
#         print(f"\n{horizon.upper()} Horizon:")
#         print("-"*90)

#         horizon_data = combined_df[combined_df['Horizon'] == horizon]
#         pivot = horizon_data.pivot_table(
#             index='Dataset',
#             columns='Context_Length',
#             values='RMSE_Mean',
#             aggfunc='first'
#         )

#         print(f"\n{'Dataset':<20}", end='')
#         for ctx in context_lengths:
#             if ctx in pivot.columns:
#                 print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
#         print()
#         print("-"*90)

#         for dataset in pivot.index:
#             print(f"{dataset:<20}", end='')
#             for ctx in context_lengths:
#                 if ctx in pivot.columns:
#                     val = pivot.loc[dataset, ctx]
#                     if not np.isnan(val):
#                         print(f"{val:.2f}".ljust(15), end='')
#                     else:
#                         print("N/A".ljust(15), end='')
#             print()

#     print(f"\n{'='*90}")
#     print("MAE COMPARISON ACROSS DATASETS")
#     print(f"{'='*90}")

#     for horizon in ['15min', '30min', '60min', '90min']:
#         print(f"\n{horizon.upper()} Horizon:")
#         print("-"*90)

#         horizon_data = combined_df[combined_df['Horizon'] == horizon]
#         pivot = horizon_data.pivot_table(
#             index='Dataset',
#             columns='Context_Length',
#             values='MAE_Mean',
#             aggfunc='first'
#         )

#         print(f"\n{'Dataset':<20}", end='')
#         for ctx in context_lengths:
#             if ctx in pivot.columns:
#                 print(f"{ctx}({ctx*5/60:.0f}h)".ljust(15), end='')
#         print()
#         print("-"*90)

#         for dataset in pivot.index:
#             print(f"{dataset:<20}", end='')
#             for ctx in context_lengths:
#                 if ctx in pivot.columns:
#                     val = pivot.loc[dataset, ctx]
#                     if not np.isnan(val):
#                         print(f"{val:.2f}".ljust(15), end='')
#                     else:
#                         print("N/A".ljust(15), end='')
#             print()

# print("\n" + "="*90)
# print("EVALUATION COMPLETE FOR ALL DATASETS!")
# print("="*90)

PROCESSING MULTIPLE DATASETS

[1/1] PROCESSING DATASET: 1_Hall2018
Found 53 patient files

  [1/53] Processing 1636-69-001...
  Patient: 1636-69-001
    Total points: 371, Sequences: 1
    Context 12: 342 predictions
    Context 48: 306 predictions
    Context 96: 258 predictions
    Context 144: 210 predictions
    Context 192: 162 predictions
    Context 288: 66 predictions

  [2/53] Processing 1636-69-026...
  Patient: 1636-69-026
    Total points: 366, Sequences: 2
    Context 12: 308 predictions
    Context 48: 242 predictions
    Context 96: 194 predictions
    Context 144: 146 predictions
    Context 192: 98 predictions
    Context 288: 2 predictions

  [3/53] Processing 1636-69-028...
  Patient: 1636-69-028
    Total points: 370, Sequences: 3
    Context 12: 305 predictions
    Context 48: 233 predictions
    Context 96: 168 predictions
    Context 144: 120 predictions
    Context 192: 72 predictions
    Context 288: No predictions (insufficient data)

  [4/53] Processing 1636-